In [3]:
from genetic_algorithm import *

## hexadecimal generator

In [ ]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 1
nr_of_genes = 6
nr_individuals = 6

res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## Sumuj duble

In [ ]:
dic = {}
for nr in res:
    dic[nr] = {}
    for i in res[nr]:
        total = dic.get(i.differ_neuron, 0) + i.weight 
        dic[nr][i.differ_neuron] = total

## grupuj 

In [2]:
# wybierz wszystkie in_mid

nr_neuronu = 4
for i in dic[nr_neuronu]:
    if 'in' in i and 'mid' in i:
        print('in_mid', i)
        
for i in dic[nr_neuronu]:
    if 'mid' in i and 'mid' in i or 'out' in i:
        print('mid+mid or out', i)